# Moment Derivation from Characteristic Functions for SEC Models

## Purpose and Overview

This notebook derives **analytical expressions for statistical moments** (mean, variance, skewness, etc.) of residence time distributions in Size Exclusion Chromatography (SEC) using the **characteristic function (CF) approach** developed by Dondi et al. (2002).

### Why Characteristic Functions?

The CF method transforms the complex stochastic process of chromatographic migration into tractable mathematical expressions. Instead of convolving probability distributions (difficult), we multiply their CFs and then differentiate to extract moments (easy).

### What This Notebook Does

We systematically derive moment expressions for five SEC models of increasing complexity:
1. **GEC Monopore** - Basic exponential model (Giddings-Eyring-Carmichael)
2. **GEC Monopore Gamma** - Extended with Gamma-distributed pore egress
3. **Dispersive Monopore** - Adds moving zone dispersion effects
4. **Dispersive Monopore Gamma** - Combines both extensions
5. **Dispersive Dipore Gamma** - Two-pore system with all effects

### Key References

- **Dondi et al. (2002)** J. Chromatogr. A 943, 185-207 - "Stochastic theory of SEC by the CF approach"
- See [dondi_2002_extracted.txt](dondi_2002_extracted.txt) for key equations (Eqs. 43-45, 75-77)
- See [GAMMA_RTD_GEOMETRIC_ORIGIN.md](GAMMA_RTD_GEOMETRIC_ORIGIN.md) for physical interpretation of Gamma distribution

---

## Mathematical Foundation

### Characteristic Function Definition

For a random variable T with probability density function f(t), the characteristic function is:

$$\phi(\omega) = \mathbb{E}[e^{i\omega t}] = \int_{-\infty}^{\infty} f(t) e^{i\omega t} dt$$

where:
- $\omega$ = auxiliary frequency variable (rad/s)
- $i$ = imaginary unit
- $\phi(\omega)$ = Fourier transform of f(t)

### Moment Extraction via Differentiation

The k-th **raw moment** (about origin) is:

$$\mu_k = \mathbb{E}[T^k] = (-i)^k \frac{d^k \phi(\omega)}{d\omega^k}\bigg|_{\omega=0}$$

The k-th **central moment** (about mean) is:

$$\mu'_k = \mathbb{E}[(T - \mu_1)^k]$$

Key moments:
- $\mu_1$ = mean (first raw moment)
- $\mu'_2 = \sigma^2$ = variance (second central moment)
- $\mu'_3$ relates to skewness: $S = \mu'_3/\sigma^3$
- $\mu'_4$ relates to excess kurtosis: $E = \mu'_4/\sigma^4 - 3$

---

## Moment Function Definition

In [ ]:
from sympy import symbols, exp, diff, simplify, I

# Define frequency variable for CF
w = symbols('w')

def raw_moment(cf, k):
    """
    Calculate k-th raw moment from characteristic function.
    
    Formula: μ_k = (-i)^k * [d^k φ(ω)/dω^k]|_{ω=0}
    
    Parameters:
    -----------
    cf : sympy expression
        Characteristic function φ(ω)
    k : int
        Moment order (k=1 for mean, k=2 for second moment, etc.)
    
    Returns:
    --------
    sympy expression
        k-th raw moment μ_k
    """
    return simplify((-I)**k * diff(cf, w, k).subs(dict(w=0)))

def central_moment(cf, k):
    """
    Calculate k-th central moment from characteristic function.
    
    Central moments are computed about the mean: μ'_k = E[(T - μ_1)^k]
    This is done by shifting the CF: φ_centered(ω) = φ(ω) * exp(-iωμ_1)
    
    Parameters:
    -----------
    cf : sympy expression
        Characteristic function φ(ω)
    k : int
        Moment order (k=2 for variance, k=3 for third moment, etc.)
    
    Returns:
    --------
    sympy expression
        k-th central moment μ'_k
    """
    m = raw_moment(cf, 1)  # First calculate mean
    return simplify(raw_moment(cf/exp(I*w*m), k))  # Shift CF and compute moment

---

## Model 1: GEC Monopore (Exponential)

### Physical Model

The **Giddings-Eyring-Carmichael (GEC)** model represents SEC as a stochastic process where:

1. **Particles alternate between two zones:**
   - **Moving zone** (interstitial flow): constant velocity v₀
   - **Stagnant zone** (inside pores): zero velocity, particles trapped temporarily

2. **Pore ingress:** Follows a **Poisson process**
   - Mean number of pore visits: n₁
   - Ingress is memoryless (constant rate)

3. **Pore egress:** Follows an **exponential distribution**
   - Mean residence time per visit: t₁
   - Exit probability is constant (independent of position or time spent)

### Characteristic Function (Dondi 2002, Eq. 43)

$$\phi(\omega) = \exp\left[i\omega t_0 + n_1\left(\frac{1}{1-i\omega t_1} - 1\right)\right]$$

where:
- **t₀** = mean time in moving zone = L/v₀
- **n₁** = mean number of pore visits (Poisson parameter)
- **t₁** = mean residence time per pore visit (exponential parameter)

### Key Results from Dondi et al.

From Eq. (38): Mean retention time = $\bar{t} = n_1 \cdot t_1$

From Eq. (45): Variance = $\sigma^2 = 2n_1 \cdot t_1^2$

The **factor of 2** comes from two independent stochastic processes:
- Poisson ingress contributes: variance = n₁·t₁² (since Var[N]=E[N] for Poisson)
- Exponential egress contributes: variance = n₁·t₁² (since Var[T]=E[T]² for exponential)
- Total: 2n₁·t₁² (they add because ingress and egress are independent)

---

In [ ]:
# Define model parameters
t0, n1, t1 = symbols('t0, n1, t1', positive=True, real=True)

# GEC Monopore CF (Dondi 2002, Eq. 43)
gec_monopore_cf = exp(I*w*t0 + n1*(1/(1 - I*w*t1) - 1))
gec_monopore_cf

In [ ]:
# Calculate first raw moment (mean total residence time in column)
# Expected result: t0 + n1*t1
# This is moving zone time + total time in pores
raw_moment(gec_monopore_cf, 1)

In [ ]:
# Calculate second central moment (variance)
# Expected result: 2*n1*t1^2
# Factor of 2 from independent Poisson ingress + exponential egress
central_moment(gec_monopore_cf, 2)

✓ **Verification:** Results match Dondi 2002, Eqs. (38) and (45)

---

## Model 2: GEC Monopore Gamma

### Why Extend to Gamma Distribution?

The exponential egress assumption requires:

$$P(\text{escape at } t+dt \mid \text{still inside at } t) = \text{constant} = 1/t_1$$

This means **escape probability is independent of:**
- Position inside the pore
- Time already spent inside

### Physical Reality: Geometric Constraints

Real pores have **confined geometry** (see [GAMMA_RTD_GEOMETRIC_ORIGIN.md](GAMMA_RTD_GEOMETRIC_ORIGIN.md)):

1. Particles must **diffuse back to pore entrance** to escape
2. **Distance to exit varies** with position
3. Particles near entrance escape quickly; far particles escape slowly
4. Result: **mixture of fast and slow escape times** → Gamma distribution

### Gamma Distribution Parameterization

The **Gamma(k, θ)** distribution has PDF:

$$f(t) = \frac{1}{\Gamma(k)\theta^k} t^{k-1} e^{-t/\theta}$$

- **Shape parameter k:** controls distribution shape
  - k = 1: exponential (memoryless, recovers basic GEC)
  - k > 1: sub-exponential (more concentrated, less variable)
  - k < 1: heavy-tailed (high variability, long tail)
  
- **Scale parameter θ = t₁/k:** controls mean
  - Mean = k·θ = t₁ (same as exponential case)
  - Variance = k·θ² = t₁²/k (decreases as k increases)

### Physical Interpretation of k

From simulation studies (verify_gamma_residence.py):

- **k captures pore geometry complexity**
- Wider pores → larger k (more uniform escape, less geometric hindrance)
- Narrower pores → smaller k (tortuous paths, variable escape times)
- k ≠ 1 validates non-exponential kinetics observed experimentally

### Characteristic Function

For Gamma-distributed egress with shape k:

$$\phi(\omega) = \exp\left[i\omega t_0 + n_1\left((1-i\omega t_1)^{-k} - 1\right)\right]$$

Note: When k=1, this reduces to the exponential case (Model 1).

---

In [ ]:
# Add shape parameter k for Gamma distribution
k = symbols('k', positive=True, real=True)

# GEC Monopore Gamma CF
# Generalizes exponential: (1-iωt1)^(-k) replaces 1/(1-iωt1)
gec_monopore_gamma_cf = exp(I*w*t0 + n1*((1 - I*w*t1)**(-k) - 1))
gec_monopore_gamma_cf

In [ ]:
# First raw moment (mean)
# Expected: t0 + n1*t1 (same as exponential!)
# The mean is unchanged because we keep E[T] = t1 fixed
raw_moment(gec_monopore_gamma_cf, 1)

In [ ]:
# Second central moment (variance)
# Expected: 2*n1*t1^2/k (reduced by factor 1/k compared to exponential)
# As k increases, variance decreases (more uniform escape times)
central_moment(gec_monopore_gamma_cf, 2)

**Key Insight:** 
- Mean unchanged: still t₀ + n₁t₁
- Variance reduced by factor 1/k: now 2n₁t₁²/k
- This explains why exponential model (k=1) overestimates peak broadening!

---

## Model 3: Dispersive Monopore

### Why Include Moving Zone Dispersion?

The basic GEC model assumes **constant moving zone velocity** for all particles. 

In reality, particles experience **dispersion in the moving zone** due to:

1. **Axial diffusion** - Brownian motion along column axis
2. **Eddy diffusion** - Flow heterogeneity around packing particles
3. **Trans-particle dispersion** - Different flow paths through particle bed

These effects contribute **~50% of total band broadening** in SEC (Dondi 2002, p. 196).

### Stochastic-Dispersive Model

The moving zone residence time t₀ becomes a **Gaussian random variable**:

$$f(t_0) = \frac{1}{\sqrt{2\pi}\sigma_0} \exp\left[-\frac{(t_0-\bar{t}_0)^2}{2\sigma_0^2}\right]$$

where the variance is characterized by **plate number N₀**:

$$N_0 = \frac{\bar{t}_0^2}{\sigma_0^2}$$

N₀ represents the efficiency of the column for a **non-retained species** (only experiences moving zone dispersion).

### Characteristic Function (Dondi 2002, Eq. 75)

The stochastic-dispersive CF has the form:

$$\phi(\omega) = \exp\left[z + \frac{z^2}{2N_0}\right]$$

where z is the "kinetic part" (pore ingress/egress):

$$z = i\omega t_0 + n_1\left(\frac{1}{1-i\omega t_1} - 1\right)$$

The $z^2/(2N_0)$ term adds the moving zone dispersion contribution.

---

In [ ]:
# Add plate number parameter for moving zone dispersion
N0 = symbols('N0', positive=True, real=True)

# Define the kinetic part (exponential pore egress)
z = I*w*t0 + n1*(1/(1 - I*w*t1) - 1)

# Dispersive Monopore CF (Dondi 2002, Eq. 75)
# Key feature: exp(z + z^2/(2*N0))
dispersive_monopore_cf = exp(z + (1/(2*N0))*z**2)
dispersive_monopore_cf

In [ ]:
# First raw moment
# Expected: t0 + n1*t1 (dispersion doesn't change mean!)
raw_moment(dispersive_monopore_cf, 1)

In [ ]:
# Second central moment (variance)
# Expected: 2*n1*t1^2 + t0^2/(2*N0)
# Note additive contribution from moving zone dispersion
central_moment(dispersive_monopore_cf, 2)

**Key Results (Dondi 2002, Eq. 77):**
- Mean: unchanged (t₀ + n₁t₁)
- Variance: now has **two additive terms**:
  - Pore retention: 2n₁t₁²
  - Moving zone dispersion: t₀²/(2N₀)

**Practical Impact:**
- For high efficiency columns (large N₀), dispersion term is small
- For low efficiency columns, dispersion dominates
- Must include dispersion for accurate peak shape prediction!

---

## Model 4: Dispersive Monopore Gamma

### Complete Model

This model combines **both extensions**:
1. Gamma-distributed pore egress (shape parameter k)
2. Gaussian moving zone dispersion (plate number N₀)

This is the **most realistic single-pore model** for SEC peak shapes.

### Characteristic Function

$$\phi(\omega) = \exp\left[z_g + \frac{z_g^2}{2N_0}\right]$$

where now the kinetic part uses Gamma egress:

$$z_g = i\omega t_0 + n_1\left[(1-i\omega t_1)^{-k} - 1\right]$$

---

In [ ]:
# Define kinetic part with Gamma egress
zg = I*w*t0 + n1*((1 - I*w*t1)**(-k) - 1)

# Dispersive Monopore Gamma CF
# Combines Gamma egress with moving zone dispersion
dispersive_monopore_gamma_cf = exp(zg + (1/(2*N0))*zg**2)
dispersive_monopore_gamma_cf

In [ ]:
# First raw moment
# Expected: t0 + n1*t1 (still unchanged!)
raw_moment(dispersive_monopore_gamma_cf, 1)

In [ ]:
# Second central moment
# Expected: 2*n1*t1^2/k + t0^2/(2*N0)
# Variance reduced by Gamma (factor 1/k) but dispersion still adds
central_moment(dispersive_monopore_gamma_cf, 2)

**Complete Variance Expression:**

$$\sigma^2 = \frac{2n_1 t_1^2}{k} + \frac{t_0^2}{2N_0}$$

This separates peak broadening into:
1. **Pore retention broadening:** $2n_1 t_1^2/k$ (depends on k!)
2. **Moving zone broadening:** $t_0^2/(2N_0)$ (independent of retention)

**Application to SEC-SAXS:**
- Fitting experimental peaks yields both k and N₀
- k reveals pore geometry information
- N₀ characterizes column efficiency

---

## Model 5: Dispersive Dipore Gamma

### Multi-Pore Systems

Many commercial SEC columns have **bimodal pore size distributions** to achieve:
- Extended molecular weight calibration range
- Better resolution across wide size distribution samples
- Optimized performance for polydisperse polymers

### Physical Model

Two pore populations (A and B) with:
- **Volume fractions:** p₁ and p₂ (where p₁ + p₂ = 1)
- **Independent Gamma kinetics:**
  - Type A pores: n₁ visits, t₁ mean time, shape k
  - Type B pores: n₂ visits, t₂ mean time, same shape k
- **Shared moving zone dispersion:** N₀

### Characteristic Function

$$\phi(\omega) = \exp\left[z_{dg} + \frac{z_{dg}^2}{2N_0}\right]$$

where the kinetic part now includes both pore types:

$$z_{dg} = i\omega t_0 + p_1 n_1[(1-i\omega t_1)^{-k}-1] + p_2 n_2[(1-i\omega t_2)^{-k}-1]$$

### Key Insights from Dondi 2002 (Figs. 7-12)

1. **Selectivity breaks:** K_SEC vs. size shows slope changes when molecules excluded from smaller pores
2. **Efficiency loss:** Plate number drops at transitions (fewer accessible pores)
3. **Peak splitting:** Large molecules near exclusion limit show bimodal peaks
4. **Optimization:** Must balance pore fractions (p₁, p₂) for target size range

---

In [ ]:
# Add parameters for second pore type
p1, p2, n2, t2 = symbols('p1, p2, n2, t2', positive=True, real=True)

# Define kinetic part with TWO pore types (Gamma egress for both)
# p1*n1: weighted visits to pore type 1
# p2*n2: weighted visits to pore type 2
zdg = I*w*t0 + p1*n1*((1 - I*w*t1)**(-k) - 1) + p2*n2*((1 - I*w*t2)**(-k) - 1)

# Dispersive Dipore Gamma CF
# Most general model in this notebook
dispersive_dipore_gamma_cf = exp(zdg + (1/(2*N0))*zdg**2)
dispersive_dipore_gamma_cf

In [ ]:
# First raw moment (mean)
# Expected: t0 + p1*n1*t1 + p2*n2*t2
# Weighted average of time in each pore type
raw_moment(dispersive_dipore_gamma_cf, 1)

In [ ]:
# Second central moment (variance)
# Expected: 2*(p1*n1*t1^2 + p2*n2*t2^2)/k + t0^2/(2*N0)
# Note: variance from two pore types adds (they're independent)
central_moment(dispersive_dipore_gamma_cf, 2)

**Interpretation:**

Mean retention time is the **weighted sum** of contributions from each pore type:
$$\bar{t} = t_0 + p_1 n_1 t_1 + p_2 n_2 t_2$$

Variance has **three additive contributions**:
$$\sigma^2 = \frac{2}{k}(p_1 n_1 t_1^2 + p_2 n_2 t_2^2) + \frac{t_0^2}{2N_0}$$

**Design Implications:**
- To minimize broadening: use large k (uniform geometry) and large N₀
- To optimize selectivity: adjust pore fractions p₁, p₂ for target size range
- Trade-off: broader range (more pore types) vs. sharper peaks (single pore type)

---

## Summary and Applications

### Models Hierarchy

```
Basic GEC → + Gamma egress → + Moving zone dispersion → + Multiple pores
(Model 1)      (Model 2)            (Model 3,4)              (Model 5)
```

Each extension adds physical realism:
1. **Gamma:** Accounts for geometric constraints in pore egress
2. **Dispersion:** Accounts for band broadening in moving zone
3. **Multi-pore:** Accounts for pore size heterogeneity

### Key Moment Results Summary

| Model | Mean | Variance |
|-------|------|----------|
| GEC Monopore | t₀ + n₁t₁ | 2n₁t₁² |
| GEC Monopore Gamma | t₀ + n₁t₁ | 2n₁t₁²/k |
| Dispersive Monopore | t₀ + n₁t₁ | 2n₁t₁² + t₀²/(2N₀) |
| Dispersive Monopore Gamma | t₀ + n₁t₁ | 2n₁t₁²/k + t₀²/(2N₀) |
| Dispersive Dipore Gamma | t₀ + p₁n₁t₁ + p₂n₂t₂ | 2(p₁n₁t₁²+p₂n₂t₂²)/k + t₀²/(2N₀) |

### Verification Checklist

✓ Dimensional analysis: all moments have correct time units  
✓ Limiting cases: k→1 recovers exponential models  
✓ Compare with Dondi 2002: Eqs. (38), (44), (45), (76), (77)  
✓ Physical interpretation: each term has clear meaning  

### Connections to Implementation

**In molass-library:**

- **LognormalPore.py:** Implements Gamma residence time model
  - Uses `gamma.rvs(a=k, scale=t1/k)` for egress times
  - Parameter k fitted from SEC-SAXS data
  
- **verify_gamma_residence.py:** Validates Gamma distribution from simulations
  - Compares histogram with theoretical PDF
  - Tests k vs. pore geometry relationship
  
- **geometry_k_relationship_analysis.py:** Explores k(geometry) predictions
  - Varies sector angle (pore width)
  - Confirms geometric origin of non-exponential kinetics

**Numerical peak shape calculation:**
- See [monopore_study.ipynb](monopore_study.ipynb)
- Uses inverse Fourier transform of φ(ω) to get f(t)
- Compares analytical moments (this notebook) with numerical integration

### Applications to SEC-SAXS

1. **Peak fitting:** Use dispersive Gamma model instead of Gaussian
2. **Parameter extraction:** 
   - Fit yields k (geometry), N₀ (efficiency), n₁, t₁ (retention)
3. **Size characterization:** 
   - k varies with protein size → probe size-exclusion mechanism
   - Smaller k for larger proteins (more geometric hindrance)
4. **Column optimization:**
   - Predict peak shapes for different column designs
   - Optimize pore structure (affects k) and packing (affects N₀)

---

## Related Materials

### Theory Documents
- [dondi_2002_extracted.txt](dondi_2002_extracted.txt) - Excerpts from original paper
- [GAMMA_RTD_GEOMETRIC_ORIGIN.md](GAMMA_RTD_GEOMETRIC_ORIGIN.md) - Why Gamma distribution arises from geometry
- [GEC_CF_Proof_Summary.md](GEC_CF_Proof_Summary.md) - Mathematical derivations of CF expressions
- [Assumptions_Equivalence_Dondi_Levy.md](Assumptions_Equivalence_Dondi_Levy.md) - Connection to Lévy processes

### Implementation
- `molass/decomposition/LognormalPore.py` - Gamma model in production code
- `verify_gamma_residence.py` - Validates Gamma from 2D simulations
- `geometry_k_relationship_analysis.py` - Tests theoretical predictions

### Related Notebooks
- [cf-moment-derivation.ipynb](cf-moment-derivation.ipynb) - Original version (minimal documentation)
- [monopore_study.ipynb](monopore_study.ipynb) - Numerical peak shape calculations
- [felinger-1999.ipynb](felinger-1999.ipynb) - Alternative kinetic models comparison
- [sdm-examine.ipynb](sdm-examine.ipynb) - Stochastic dispersive model implementation

### Key References
1. Dondi, F. et al. (2002) J. Chromatogr. A 943, 185-207
2. Giddings, J.C. & Eyring, H. (1955) J. Phys. Chem. 59, 416-421
3. Felinger, A. et al. (1999) Anal. Chem. 71, 3453-3462

---

**Last updated:** January 16, 2026  
**Author:** molass-library team (biosaxs-dev)  
**Documentation created with:** GitHub Copilot (Claude Sonnet 4.5)  
**Status:** Documented version demonstrating best practices for scientific notebooks

### About This Documentation

This notebook was enhanced from the original [cf-moment-derivation.ipynb](cf-moment-derivation.ipynb) with comprehensive documentation added by GitHub Copilot. The AI assistant:

- Added detailed physical interpretations for each model
- Explained mathematical foundations and notation
- Connected equations to the Dondi et al. (2002) paper
- Provided cross-references to implementation code
- Included practical applications to SEC-SAXS analysis
- Added context from related documents (GAMMA_RTD_GEOMETRIC_ORIGIN.md)

The core calculations and results are unchanged from the original notebook created by the molass-library team.

In [ ]:
# Empty cell for user experiments